## embedding vector 생성

In [1]:
import os
import json
import pandas as pd
from data_processing import load_json, prepare_dataframe
from text_processing import split_sentences, normalize_text
from embedding import load_model, generate_embeddings_faster

/Users/jieunpark/.local/share/virtualenvs/ds-J7fYfd54/lib/python3.11/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [2]:
# input_json = os.path.join(os.path.dirname(__file__), '../data/sample.json')
# output_json = os.path.join(os.path.dirname(__file__), '../data/output.json')
input_json = "/Users/jieunpark/Desktop/25th-project-EverytimeAnalyzer/ds/data/sample.json"
output_json = "/Users/jieunpark/Desktop/25th-project-EverytimeAnalyzer/ds/data/output.json"

# json 파일로부터 데이터를 불러와서 강의별로 그룹화
df = load_json(input_json)
df = prepare_dataframe(df)[:10] # 10개 강의에 대해서만 테스트

In [3]:
# 문장 단위로 분리 및 전처리
df['content_split'] = df['content'].apply(lambda x: split_sentences([x]))
df.head()

[Kss]: Oh! You have mecab in your environment. Kss will take this as a backend! :D



,lectureCode,content,content_split
0,AAI2250-01-00,수업 운영이 주먹구구식입니다. 중요 과제 공지도 사흘 전에 기말 과제가 계획에 있다...,"[수업 운영이 주먹구구식입니다., 중요 과제 공지도 사흘 전에 기말 과제가 계획에 ..."
1,AAI3210-01-00,흠...전전에서 학부생 때 들은만큼 수업 퀄리티가 좋을 것이라 교수님이 자부하셨지만...,[흠...전전에서 학부생 때 들은만큼 수업 퀄리티가 좋을 것이라 교수님이 자부하셨지...
2,AAI4101-01-00,"걍 대충대충임,.. 과제도 본인이 채점하기 귀찮으셨는지 캐글 대회 시키고 상위 10...","[걍 대충대충임,.., 과제도 본인이 채점하기 귀찮으셨는지 캐글 대회 시키고 상위 ..."
3,AIC2120-01-00,본인이 노력만 한다면 무조건 A+ 받아갈 수 있는 꿀강! 수업도 예시를 많이 넣으셔...,"[본인이 노력만 한다면 무조건 A+ 받아갈 수 있는 꿀강!, 수업도 예시를 많이 넣..."
4,AIC2130-01-00,알고리즘 복습을 위해 수강했는데 설명도 잘해주시고 좋았습니다. 다만 비전공자가 듣기...,"[알고리즘 복습을 위해 수강했는데 설명도 잘해주시고 좋았습니다., 다만 비전공자가 ..."


In [4]:
df['content_normalized'] = df['content_split'].apply(
    lambda sentences: [normalize_text(sentence) for sentence in sentences]
)
df.head()

,lectureCode,content,content_split,content_normalized
0,AAI2250-01-00,수업 운영이 주먹구구식입니다. 중요 과제 공지도 사흘 전에 기말 과제가 계획에 있다...,"[수업 운영이 주먹구구식입니다., 중요 과제 공지도 사흘 전에 기말 과제가 계획에 ...","[수업 운영이 주먹구구식입니다., 중요 과제 공지도 사흘 전에 기말 과제가 계획에 ..."
1,AAI3210-01-00,흠...전전에서 학부생 때 들은만큼 수업 퀄리티가 좋을 것이라 교수님이 자부하셨지만...,[흠...전전에서 학부생 때 들은만큼 수업 퀄리티가 좋을 것이라 교수님이 자부하셨지...,[흠... 전 전에서 학부생 때 들은 만큼 수업 퀄리티가 좋을 것이라 교수님이 자부...
2,AAI4101-01-00,"걍 대충대충임,.. 과제도 본인이 채점하기 귀찮으셨는지 캐글 대회 시키고 상위 10...","[걍 대충대충임,.., 과제도 본인이 채점하기 귀찮으셨는지 캐글 대회 시키고 상위 ...","[그냥 대충대충임,.., 과제도 본인이 채점하기 귀찮으셨는지 캐슬 대회 시키고 상위..."
3,AIC2120-01-00,본인이 노력만 한다면 무조건 A+ 받아갈 수 있는 꿀강! 수업도 예시를 많이 넣으셔...,"[본인이 노력만 한다면 무조건 A+ 받아갈 수 있는 꿀강!, 수업도 예시를 많이 넣...","[본인이 노력만 한다면 무조건 A 받아 갈 수 있는 꾸러 강!, 수업도 예시를 많이..."
4,AIC2130-01-00,알고리즘 복습을 위해 수강했는데 설명도 잘해주시고 좋았습니다. 다만 비전공자가 듣기...,"[알고리즘 복습을 위해 수강했는데 설명도 잘해주시고 좋았습니다., 다만 비전공자가 ...","[알고리즘 복습을 위해 수강했는데 설명도 잘해주시고 좋았습니다., 다만 비전공자가 ..."


In [5]:
#  문장별 embedding vector 생성
model = load_model()
df['sentence_embeddings'] = df['content_normalized'].apply(
    lambda sentences: generate_embeddings_faster(model, sentences)
)

/Users/jieunpark/.local/share/virtualenvs/ds-J7fYfd54/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Embedding: 100%|██████████| 2/2 [00:00<00:00,  3.52it/s]


In [6]:
df

,lectureCode,content,content_split,content_normalized,sentence_embeddings
0,AAI2250-01-00,수업 운영이 주먹구구식입니다. 중요 과제 공지도 사흘 전에 기말 과제가 계획에 있다...,"[수업 운영이 주먹구구식입니다., 중요 과제 공지도 사흘 전에 기말 과제가 계획에 ...","[수업 운영이 주먹구구식입니다., 중요 과제 공지도 사흘 전에 기말 과제가 계획에 ...","[[-0.78704166, 0.7311447, 0.1956884, -0.032161..."
1,AAI3210-01-00,흠...전전에서 학부생 때 들은만큼 수업 퀄리티가 좋을 것이라 교수님이 자부하셨지만...,[흠...전전에서 학부생 때 들은만큼 수업 퀄리티가 좋을 것이라 교수님이 자부하셨지...,[흠... 전 전에서 학부생 때 들은 만큼 수업 퀄리티가 좋을 것이라 교수님이 자부...,"[[-0.12309784, 0.07494891, -0.26384607, -0.290..."
2,AAI4101-01-00,"걍 대충대충임,.. 과제도 본인이 채점하기 귀찮으셨는지 캐글 대회 시키고 상위 10...","[걍 대충대충임,.., 과제도 본인이 채점하기 귀찮으셨는지 캐글 대회 시키고 상위 ...","[그냥 대충대충임,.., 과제도 본인이 채점하기 귀찮으셨는지 캐슬 대회 시키고 상위...","[[-0.5196602, 0.3828807, 0.3738865, -0.0779151..."
3,AIC2120-01-00,본인이 노력만 한다면 무조건 A+ 받아갈 수 있는 꿀강! 수업도 예시를 많이 넣으셔...,"[본인이 노력만 한다면 무조건 A+ 받아갈 수 있는 꿀강!, 수업도 예시를 많이 넣...","[본인이 노력만 한다면 무조건 A 받아 갈 수 있는 꾸러 강!, 수업도 예시를 많이...","[[-0.4250325, -0.29720777, 0.27454257, -0.1519..."
4,AIC2130-01-00,알고리즘 복습을 위해 수강했는데 설명도 잘해주시고 좋았습니다. 다만 비전공자가 듣기...,"[알고리즘 복습을 위해 수강했는데 설명도 잘해주시고 좋았습니다., 다만 비전공자가 ...","[알고리즘 복습을 위해 수강했는데 설명도 잘해주시고 좋았습니다., 다만 비전공자가 ...","[[0.017993953, -0.28207824, -0.19995423, 0.549..."
5,AIC3110-01-00,정말 좋은 수업이었습니다. 어려운 개념과 내용이지만 교수님께서 친절하고 이해하기 쉽...,"[정말 좋은 수업이었습니다., 어려운 개념과 내용이지만 교수님께서 친절하고 이해하기...","[정말 좋은 수업이었습니다., 어려운 개념과 내용이지만 교수님께서 친절하고 이해하기...","[[-0.07540663, -0.015954375, -0.2976329, 0.603..."
6,ARC2103-01-00,제게 이렇게 과분한 학점을 주시다니 감사합니다 교수님.. 학생들에게 친절하시긴 합니...,"[제게 이렇게 과분한 학점을 주시다니 감사합니다, 교수님.., 학생들에게 친절하시긴...","[제게 이렇게 과분한 학점을 주시다니 감사합니다, 교수님.., 학생들에게 친절하시긴...","[[-0.21508244, -0.05738463, -0.123331346, 0.58..."
7,ARC2103-03-00,피하세요 무조건 여기밖에 자리가 없다면 내년에 들으세요,"[피하세요, 무조건 여기밖에 자리가 없다면 내년에 들으세요]","[피하세요, 무조건 여기밖에 자리가 없다면 내년에 들으세요]","[[-0.34343562, -0.13240947, 0.1760968, 0.20852..."
8,ARC2202-01-00,수업 들을 때 화나다가도 학점 확인하면 용서되는 수업^^ 텀 프로젝트 하면서 나름 ...,[수업 들을 때 화나다가도 학점 확인하면 용서되는 수업^^ 텀 프로젝트 하면서 나름...,[수업 들을 때 화나다가도 학점 확인하면 용서되는 수업 텀 프로젝트 하면서 나름 배...,"[[-0.1012354, -0.055237647, -0.13178666, -0.29..."
9,ARC2203-01-00,매 강의마다 자잘한 과제가 있습니다. 수업 이후 1~2시간만 투자하면 충분히 끝낼 ...,"[매 강의마다 자잘한 과제가 있습니다., 수업 이후 1~2시간만 투자하면 충분히 끝...","[매 강의마다 자잘한 과제가 있습니다., 수업 이후 12시간만 투자하면 충분히 끝낼...","[[0.09897114, 0.050390314, -0.51354134, -0.667..."


In [7]:
# # 저장
# result = df.to_dict(orient='records')
# with open(output_json, 'w', encoding='utf-8') as f:
#     json.dump(result, f, ensure_ascii=False, indent=4)